In [ ]:
#take images from cam and mouse pos. look at the mouse to get where your looking for
import cv2
import pyautogui

cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        x, y = pyautogui.position()
        #print(x,y)
        img_name = str(img_counter)+'-'+str(x)+'-'+str(y)
        img_name = "opencv_frame_{}.png".format(img_name)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

In [1]:
# real time prediction

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pyautogui
from tensorflow.keras.models import load_model
import time

pyautogui.FAILSAFE = False

loaded_model = load_model('./my_eyes.h5', compile = False)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

list_heatmap = []
start_time = time.time()
nb_boucle = 0

cap = cv2.VideoCapture(0)
stop = 0
while stop == 0:
    
    #img_crop1 = np.zeros(shape=(32, 32))
    #img_crop2 = np.zeros(shape=(32, 32))
    
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    #faces = face_cascade.detectMultiScale(img, scaleFactor = 1.1, minNeighbors = 5)
    
    ex,ey,ew,eh=1,1,1,1
    
    for (ex,ey,ew,eh) in faces:
        cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(255,0,0),5)
        #print(ex,ey,eh,ew)
        milieu_crop_y = ey+eh/2
        milieu_crop_x = ex+ew/2
        distance_visage = 223/((eh+ew)/2)*0.4
        distance_visage = str(distance_visage)+'00000'
        distance_visage = float(distance_visage[0:5])
    
    image = img[ey:ey+eh, ex:ex+ew]
    eyes_array = []
    eyes = eye_cascade.detectMultiScale(image, scaleFactor = 1.1, minNeighbors = 5)
    
    for (eex,eey,eew,eeh) in eyes:
        if eey < image.shape[0]/2:
            cv2.rectangle(image,(eex,eey),(eex+eew,eey+eeh),(0,255,0),2) 
            #print(eex,eey,eeh,eew)
            eyes_array.append(eex)
            eyes_array.append(eey)
            eyes_array.append(eew)
            eyes_array.append(eeh)
            
    if len(eyes_array)== 8:        
        if eyes_array[0] > eyes_array[4]:  
            #right-left
            middle_left_x = ex+eyes_array[4]+eyes_array[6]/2
            middle_left_y = ey+eyes_array[5]+eyes_array[7]/2
            middle_right_x = ex+eyes_array[0]+eyes_array[2]/2
            middle_right_y = ey+eyes_array[1]+eyes_array[3]/2
            img_crop2 = image[eyes_array[1]:eyes_array[1]+eyes_array[3]+1, eyes_array[0]:eyes_array[0]+eyes_array[2]+1]
            img_crop1 = image[eyes_array[5]:eyes_array[5]+eyes_array[7]+1, eyes_array[4]:eyes_array[4]+eyes_array[6]+1]
        else :
            #left-right
            middle_right_x = ex+eyes_array[4]+eyes_array[6]/2
            middle_right_y = ey+eyes_array[5]+eyes_array[7]/2
            middle_left_x = ex+eyes_array[0]+eyes_array[1]/2
            middle_left_y = ey+eyes_array[2]+eyes_array[3]/2
            img_crop1 = image[eyes_array[1]:eyes_array[1]+eyes_array[3]+1, eyes_array[0]:eyes_array[0]+eyes_array[2]+1]
            img_crop2 = image[eyes_array[5]:eyes_array[5]+eyes_array[7]+1, eyes_array[4]:eyes_array[4]+eyes_array[6]+1]

        img_crop1 = img_crop1[2:img_crop1.shape[0]-2 , 2:img_crop1.shape[0]-2]
        img_crop2 = img_crop2[2:img_crop2.shape[0]-2 , 2:img_crop2.shape[0]-2]
        
        original_size_1 = img_crop1.shape[0]
        original_size_2 = img_crop2.shape[0]

        width = 32
        height = 32

        dim = (width, height) 
        img_crop1 = cv2.resize(img_crop1, dim, interpolation = cv2.INTER_AREA)
        img_crop2 = cv2.resize(img_crop2, dim, interpolation = cv2.INTER_AREA)
        
        img_crop1 = cv2.cvtColor(img_crop1, cv2.COLOR_BGR2GRAY)
        img_crop2 = cv2.cvtColor(img_crop2, cv2.COLOR_BGR2GRAY)
        
        img_2 = np.hstack((img_crop1, img_crop2))
        cv2.imshow('Eyes_frame', img_2) 
        
        img_crop1 = img_crop1/255 
        img_crop1 = img_crop1.reshape(1,32,32)
        #img_crop1 = np.expand_dims(img_crop1, axis=-1)
        #img_crop1 = img_crop1.reshape(1,32,32)
        #print(img_crop1.shape)
        
        img_crop11 = np.empty(0)
        img_crop11 = np.append(img_crop11, milieu_crop_x/640)
        img_crop11 = np.append(img_crop11, milieu_crop_y/480)
        img_crop11 = np.append(img_crop11, distance_visage)
        img_crop11 = np.append(img_crop11, middle_left_x/640)
        img_crop11 = np.append(img_crop11, middle_left_y/480)
        img_crop11 = np.append(img_crop11, original_size_1)
        img_crop11 = np.append(img_crop11, 0.5)
        img_crop11 = img_crop11.reshape(1, -1)
        #print(img_crop11.shape)
        pred1 = loaded_model.predict_on_batch([img_crop1,img_crop11])
        print(pred1)
        #nb1 = int(pred1[0 :pred1.find('-')])
        #nb2 = int(pred1[pred1.find('-')+1: len(pred1)])
        
        #img_crop2 = img_crop2.flatten()
        img_crop2 = img_crop2/255
        img_crop2 = img_crop2.reshape(1,32,32)
        
        img_crop22 = np.empty(0)
        img_crop22 = np.append(img_crop22, milieu_crop_x/640)
        img_crop22 = np.append(img_crop22, milieu_crop_y/480)
        img_crop22 = np.append(img_crop22, distance_visage)
        img_crop22 = np.append(img_crop22, middle_right_x/640)
        img_crop22 = np.append(img_crop22, middle_right_y/480)
        img_crop22 = np.append(img_crop22, original_size_2)
        img_crop22 = np.append(img_crop22, 0)
        img_crop22 = img_crop22.reshape(1, -1)
        #print(img_crop1.shape)
        pred2 = loaded_model.predict([img_crop2,img_crop22])
        print(pred2)
        #nb12 = int(pred2[0 :pred2.find('-')])
        #nb22 = int(pred2[pred1.find('-')+1: len(pred2)])
        
        x_final = ((pred1[0][0] + pred2[0][0])/2)*1920
        y_final = ((pred1[0][1]+ pred2[0][1])/2)*1080
        
        print(x_final,y_final)
        list_heatmap.append([x_final,y_final])
        
        pyautogui.moveTo(x_final, y_final) 
        nb_boucle = nb_boucle+1
    k = cv2.waitKey(30) & 0xff
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        stop=1
        break     
        
end_time = time.time()
#print(list_heatmap)
print("durée :"+str(end_time-start_time))
print(nb_boucle)
cap.release()
cv2.destroyAllWindows()

[[0.38630992 0.4653996 ]]
[[0.46636775 0.94987315]]
818.5705947875977 764.2472648620605
[[0.57570404 0.37867534]]
[[0.55013245 0.73978776]]
1080.8030319213867 603.9700412750244
[[0.54316366 0.4799201 ]]
[[0.56793594 0.8334125 ]]
1066.655616760254 709.1995811462402
[[0.10309622 0.11969257]]
[[0.21673672 0.36458465]]
307.0396327972412 261.5096980333328
[[0.08940416 0.0548412 ]]
[[0.23509754 0.34808424]]
311.5216255187988 217.57973313331604
[[0.9566407 0.3107468]]
[[1.0597605  0.20798108]]
1935.7452392578125 280.11306524276733
[[0.9662578  0.12882237]]
[[1.046512   0.18653165]]
1932.2589111328125 170.29117047786713
[[0.90848094 0.85315067]]
[[0.91459066 0.8226393 ]]
1750.1487350463867 904.9265742301941
[[0.89182025 0.833241  ]]
[[0.96402943 0.84373236]]
1781.6157531738281 905.5656051635742
[[0.8296787  0.88256884]]
[[0.9058041  0.85821384]]
1666.0634994506836 940.0226783752441
[[0.23928647 0.69968015]]
[[0.16759206 0.91851944]]
390.60338973999023 873.8277769088745
[[0.17778674 0.70139873]